<a href="https://colab.research.google.com/github/Anjali-sct27/Multilingual-BERT-Sentiment-Analysis/blob/main/Multilingual_BERT_Sentiment_Analysis_clean.ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
!pip install nltk scikit-learn pandas numpy matplotlib seaborn


In [9]:
import pandas as pd
import numpy as np
import re
import nltk
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix


In [10]:
url = "https://raw.githubusercontent.com/dD2405/Twitter_Sentiment_Analysis/master/train.csv"
data = pd.read_csv(url)

data.head()


,id,label,tweet
0,1,0,@user when a father is dysfunctional and is s...
1,2,0,@user @user thanks for #lyft credit i can't us...
2,3,0,bihday your majesty
3,4,0,#model i love u take with u all the time in ...
4,5,0,factsguide: society now #motivation


In [11]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+|#", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

data['clean_tweet'] = data['tweet'].apply(clean_text)
data.head()


LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - '/root/nltk_data'
    - '/usr/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/share/nltk_data'
    - '/usr/local/share/nltk_data'
    - '/usr/lib/nltk_data'
    - '/usr/local/lib/nltk_data'
**********************************************************************


In [ ]:
import nltk
nltk.download('stopwords')


In [ ]:
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

def clean_text(text):
    text = text.lower()
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+|#", "", text)
    text = re.sub(r"[^a-z\s]", "", text)
    words = text.split()
    words = [w for w in words if w not in stop_words]
    return " ".join(words)

data['clean_tweet'] = data['tweet'].apply(clean_text)
data.head()


In [ ]:
X = data['clean_tweet']
y = data['label']

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)


In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(X_train_tfidf, y_train)


In [ ]:
from sklearn.metrics import accuracy_score, classification_report

y_pred = model.predict(X_test_tfidf)

print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


In [ ]:
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

cm = confusion_matrix(y_test, y_pred)

sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel("Predicted Label")
plt.ylabel("Actual Label")
plt.title("Confusion Matrix")
plt.show()


In [ ]:
def predict_sentiment(text):
    text = clean_text(text)
    vector = vectorizer.transform([text])
    result = model.predict(vector)
    return "Positive 😊" if result[0] == 1 else "Negative 😠"


In [ ]:
predict_sentiment("I really love this project, it's amazing!")


In [ ]:
clean_text("I really love this project, it's amazing!")


In [ ]:
model = LogisticRegression(max_iter=1000)
model.fit(X_train_tfidf, y_train)


In [ ]:
predict_sentiment("I really love this project, it's amazing!")


In [ ]:
!pip install transformers torch


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification


In [ ]:
model_name = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)


In [ ]:
def bert_predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)

    with torch.no_grad():
        outputs = model(**inputs)

    logits = outputs.logits
    predicted_class = torch.argmax(logits, dim=1).item()

    return "Positive 😊" if predicted_class == 1 else "Negative 😠"


In [ ]:
bert_predict_sentiment("I really love this project, it's amazing!")


In [ ]:
bert_predict_sentiment("This is the worst experience ever")
bert_predict_sentiment("I hate this product")
bert_predict_sentiment("Not bad, could be better")
bert_predict_sentiment("Absolutely fantastic work!")


In [ ]:
print(bert_predict_sentiment("This is the worst experience ever"))
print(bert_predict_sentiment("I hate this product"))
print(bert_predict_sentiment("Not bad, could be better"))
print(bert_predict_sentiment("Absolutely fantastic work!"))


In [ ]:
!pip install langdetect sentencepiece


In [ ]:
from langdetect import detect
from transformers import MarianMTModel, MarianTokenizer


In [ ]:
translator_model_name = "Helsinki-NLP/opus-mt-mul-en"

translator_tokenizer = MarianTokenizer.from_pretrained(translator_model_name)
translator_model = MarianMTModel.from_pretrained(translator_model_name)


In [ ]:
def translate_to_english(text):
    inputs = translator_tokenizer(text, return_tensors="pt", padding=True, truncation=True)

    translated = translator_model.generate(**inputs)

    english_text = translator_tokenizer.decode(translated[0], skip_special_tokens=True)
    return english_text


In [ ]:
def multilingual_sentiment(text):
    try:
        lang = detect(text)
    except:
        lang = "unknown"

    # If not English, translate
    if lang != "en":
        translated_text = translate_to_english(text)
    else:
        translated_text = text

    sentiment = bert_predict_sentiment(translated_text)

    return {
        "original_text": text,
        "detected_language": lang,
        "translated_text": translated_text,
        "sentiment": sentiment
    }


In [ ]:
multilingual_sentiment("I really love this project")


In [ ]:
multilingual_sentiment("ഈ പ്രോജക്ട് വളരെ നല്ലതാണ്")


In [ ]:
multilingual_sentiment("यह अनुभव बहुत खराब था")


In [ ]:
multilingual_sentiment("இந்த செயலி அருமையாக உள்ளது")


In [ ]:
multilingual_sentiment("இந்த சேவை மிகவும் மோசமாக உள்ளது")


In [ ]:
multilingual_sentiment("ഈ അനുഭവം തീരെ മോശമായിരുന്നു")


In [ ]:
multilingual_sentiment("ఈ యాప్ చాలా ఉపయోగకరంగా ఉంది")


In [ ]:
multilingual_sentiment("Cette application est vraiment géniale")


In [ ]:
multilingual_sentiment("Esta aplicación es terrible")


In [ ]:
multilingual_sentiment("Diese App ist sehr nützlich")


In [ ]:
multilingual_sentiment("このアプリは最悪です")


In [12]:
multilingual_sentiment("이 앱은 정말 좋아요")


NameError: name 'multilingual_sentiment' is not defined

In [13]:
globals().keys()


dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', '_exit_code', '_i2', 'pd', 'np', 're', 'nltk', 'plt', 'sns', 'train_test_split', 'TfidfVectorizer', 'LogisticRegression', 'accuracy_score', 'classification_report', 'confusion_matrix', '_i3', 'url', 'data', '_3', '_i4', 'stopwords', '_i5', '_i6', '_i7', '_i8', '_i9', '_i10', '_10', '_i11', '_i12', '_i13'])

In [14]:
!pip install transformers torch langdetect sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 16.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=ab61de31cfe928ff748ee7f63c104f41af85f5a2afbe503d925095e7f8a81eec
  Stored in directory: /root/.cache/pip/wheels/c1/67/88/e844b5b022812e15a52e4eaa38a1e709e99f06f6639d7e3ba7
Successfully built langdetect


In [15]:
import torch
from langdetect import detect
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import MarianMTModel, MarianTokenizer


In [16]:
bert_model_name = "distilbert-base-uncased-finetuned-sst-2-english"

tokenizer = AutoTokenizer.from_pretrained(bert_model_name)
model = AutoModelForSequenceClassification.from_pretrained(bert_model_name)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/629 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [17]:
def bert_predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    with torch.no_grad():
        outputs = model(**inputs)
    pred = torch.argmax(outputs.logits, dim=1).item()
    return "Positive" if pred == 1 else "Negative"


In [18]:
translator_name = "Helsinki-NLP/opus-mt-mul-en"

translator_tokenizer = MarianTokenizer.from_pretrained(translator_name)
translator_model = MarianMTModel.from_pretrained(translator_name)


tokenizer_config.json:   0%|          | 0.00/44.0 [00:00<?, ?B/s]

source.spm:   0%|          | 0.00/707k [00:00<?, ?B/s]

target.spm:   0%|          | 0.00/791k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

/usr/local/lib/python3.12/dist-packages/transformers/models/marian/tokenization_marian.py:175: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


pytorch_model.bin:   0%|          | 0.00/310M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/310M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/293 [00:00<?, ?B/s]

In [19]:
def translate_to_english(text):
    inputs = translator_tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    translated = translator_model.generate(**inputs)
    return translator_tokenizer.decode(translated[0], skip_special_tokens=True)


In [20]:
def multilingual_sentiment(text):
    try:
        lang = detect(text)
    except:
        lang = "unknown"

    if lang == "en":
        translated = text
    else:
        translated = translate_to_english(text)

    sentiment = bert_predict_sentiment(translated)

    return {
        "original_text": text,
        "detected_language": lang,
        "translated_text": translated,
        "sentiment": sentiment
    }


In [21]:
multilingual_sentiment("이 앱은 정말 좋아요")


{'original_text': '이 앱은 정말 좋아요',
 'detected_language': 'ko',
 'translated_text': 'Whoa, whoa, whoa, whoa, whoa.',
 'sentiment': 'Negative'}

In [22]:
globals().keys()


dict_keys(['__name__', '__doc__', '__package__', '__loader__', '__spec__', '__builtin__', '__builtins__', '_ih', '_oh', '_dh', 'In', 'Out', 'get_ipython', 'exit', 'quit', '_', '__', '___', '_i', '_ii', '_iii', '_i1', '_exit_code', '_i2', 'pd', 'np', 're', 'nltk', 'plt', 'sns', 'train_test_split', 'TfidfVectorizer', 'LogisticRegression', 'accuracy_score', 'classification_report', 'confusion_matrix', '_i3', 'url', 'data', '_3', '_i4', 'stopwords', '_i5', '_i6', '_i7', '_i8', '_i9', '_i10', '_10', '_i11', '_i12', '_i13', '_13', '_i14', '_i15', 'torch', 'detect', 'AutoTokenizer', 'AutoModelForSequenceClassification', 'MarianMTModel', 'MarianTokenizer', '_i16', 'bert_model_name', 'tokenizer', 'model', '_i17', 'bert_predict_sentiment', '_i18', 'translator_name', 'translator_tokenizer', 'translator_model', '_i19', 'translate_to_english', '_i20', 'multilingual_sentiment', '_i21', '_21', '_i22'])

In [23]:
multilingual_sentiment("This app is very helpful and easy to use")


{'original_text': 'This app is very helpful and easy to use',
 'detected_language': 'en',
 'translated_text': 'This app is very helpful and easy to use',
 'sentiment': 'Positive'}

In [24]:
multilingual_sentiment("Esta aplicación es muy útil")


{'original_text': 'Esta aplicación es muy útil',
 'detected_language': 'es',
 'translated_text': 'This application is very useful',
 'sentiment': 'Positive'}

In [25]:
multilingual_sentiment("Cette application est vraiment excellente")


{'original_text': 'Cette application est vraiment excellente',
 'detected_language': 'fr',
 'translated_text': 'This application is really excellent',
 'sentiment': 'Positive'}

In [26]:
multilingual_sentiment("यह ऐप बहुत अच्छा है")


{'original_text': 'यह ऐप बहुत अच्छा है',
 'detected_language': 'hi',
 'translated_text': "It's very good.",
 'sentiment': 'Positive'}

In [27]:
multilingual_sentiment("இந்த செயலி மிகவும் பயனுள்ளதாக உள்ளது")


{'original_text': 'இந்த செயலி மிகவும் பயனுள்ளதாக உள்ளது',
 'detected_language': 'ta',
 'translated_text': 'This operation is most useful',
 'sentiment': 'Positive'}

In [28]:
multilingual_sentiment("ഈ അനുഭവം വളരെ മോശമായിരുന്നു")


{'original_text': 'ഈ അനുഭവം വളരെ മോശമായിരുന്നു',
 'detected_language': 'ml',
 'translated_text': 'This experience was very bad.',
 'sentiment': 'Negative'}

In [29]:
multilingual_sentiment("이 앱은 최악이에요")


{'original_text': '이 앱은 최악이에요',
 'detected_language': 'ko',
 'translated_text': 'Whoa, whoa, whoa, whoa, whoa.',
 'sentiment': 'Negative'}

In [30]:
multilingual_sentiment("このアプリはとても便利です")


{'original_text': 'このアプリはとても便利です',
 'detected_language': 'ja',
 'translated_text': 'This is very easy.',
 'sentiment': 'Positive'}

In [31]:
multilingual_sentiment("هذا التطبيق سيئ للغاية")


{'original_text': 'هذا التطبيق سيئ للغاية',
 'detected_language': 'ar',
 'translated_text': 'This application is very bad.',
 'sentiment': 'Negative'}